In [1]:
!pip install tensorflow

In [2]:
!pip install keras

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Convolution2D, Conv1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling1D
from keras import backend as K
from keras import backend
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import time
import os
import psutil
import csv
from itertools import repeat
from PIL import Image
from numpy import asarray
from collections import defaultdict

In [5]:
dfDS = pd.read_csv('/content/drive/MyDrive/master project/dataset_fl.csv')

X_full = dfDS.iloc[:, 1:len(dfDS.columns)].values

Y_full = dfDS["PTRAN_QMT"].values

xTrain, xTest, yTrain, yTest = train_test_split(X_full, Y_full, test_size=0.10, random_state=123)

In [6]:
X_full.shape

(76101, 4)

In [7]:
yTrain.shape

(68490,)

In [8]:
yTest.shape

(7611,)

In [9]:
algoName='ANN' #CNN, ANN, DNN

xTrain = xTrain.astype('float32')
xTest = xTest.astype('float32')
xTrain = xTrain / 255.
xTest = xTest / 255.


outputClasses=1
#One hot encoding
yTrain = np.array(yTrain)
yTest = np.array(yTest)
print("xTrain", xTrain.shape, "yTrain", yTrain.shape)
print("xTest", xTest.shape, "yTest", yTest.shape)

# FOR TEST SPLIT
xServer, xClients, yServer, yClients = train_test_split(xTrain, yTrain, test_size=0.80,random_state=523)

def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    recall=recall_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("Recall : {}".format(recall))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, recall, f1Score

verbose, epochs, batch_size = 0, 20, 64
activationFun='relu'
optimizerName='Adam'
def createDeepModel():
    model = Sequential()

    if(algoName=='ANN'):
        model.add(Dense(200, input_dim=X_full.shape[1], activation=activationFun))
        model.add(Flatten())
        model.add(Dense(outputClasses, activation='softmax'))
        model.compile(loss='mean_squared_error', optimizer=optimizerName, metrics=['accuracy'])

    elif(algoName=='DNN'):
        model.add(Dense(200, input_dim=X_full.shape[1], activation=activationFun))
        model.add(Dense(100, activation=activationFun))
        model.add(Dense(50, activation=activationFun))
        model.add(Dense(25,  activation=activationFun))
        model.add(Flatten())
        model.add(Dense(outputClasses, activation='softmax'))
        model.compile(loss='mean_squared_error', optimizer=optimizerName, metrics=['accuracy'])
    return model

def predictTestData(yPredict, yTest):
    #Converting predictions to label
    print("yPredict",len(yPredict))
    pred = list()
    for i in range(len(yPredict)):
        pred.append(np.argmax(yPredict[i]))
    #Converting one hot encoded test label to label
    test = list()
    for i in range(len(yTest)):
        test.append(np.argmax(yTest[i]))
    return my_metrics(test, pred)

def sumOfWeights(weights):
    return sum(map(sum, weights))

def getWeights(model):
    allLayersWeights=deepModel.get_weights()
    return allLayersWeights

# Initially train central deep model
deepModel=createDeepModel()

xTrain (68490, 4) yTrain (68490,)
xTest (7611, 4) yTest (7611,)


In [11]:
numOfIterations=10
numOfClients=10 # 10, 15, 20, 25, 30, 35, 40, 45, 50
modelLocation='/content/drive/MyDrive/master project/'+"Model/"+str(algoName)+"_Sync_users_"+str(numOfClients)+"_"+activationFun+"_"+optimizerName+"_FL_Model.h5"
accList, precList, recallList, f1List = [], [], [], []

deepModelAggWeights=[]
firstClientFlag=True

def updateServerModel(clientModel, clientModelWeight):
    global firstClientFlag
    for ind in range(len(clientModelWeight)):
        if(firstClientFlag==True):
            deepModelAggWeights.append(clientModelWeight[ind])
        else:
            deepModelAggWeights[ind]=(deepModelAggWeights[ind]+clientModelWeight[ind])

def updateClientsModels():
    global clientsModelList
    global deepModel
    clientsModelList.clear()
    for clientID in range(numOfClients):
        m = keras.models.clone_model(deepModel)
        m.set_weights(deepModel.get_weights())
        clientsModelList.append(m)

# ----- 1. Train central model initially -----
def trainInServer():
    deepModel.fit(xServer, yServer, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # deepModel.fit(X_full, Y_full, epochs=epochs, batch_size=batch_size, verbose=verbose)
    deepModel.save(modelLocation)
trainInServer()
# ------- 2. Separate clients data into lists ----------
xClientsList=[]
yClientsList=[]
clientsModelList=[]
# Adjust the interval for client data based on the number of groups
clientDataInterval = num_groups // numOfClients
lastLowerBound = 0

# Create a dictionary to group data by the second element in each sub-array of xClients
grouped_xClients = defaultdict(list)
grouped_yClients = defaultdict(list)

# for clientID in range(numOfClients):
#     xClientsList.append(xClients[lastLowerBound : lastLowerBound+clientDataInterval])
#     yClientsList.append(yClients[lastLowerBound : lastLowerBound+clientDataInterval])
#     print(xClientsList)
#     model=load_model(modelLocation)
#     clientsModelList.append(model)
#     lastLowerBound+=clientDataInterval

for x, y in zip(xClients, yClients):
    key = x[1]  # Assuming the second element is used for grouping
    grouped_xClients[key].append(x)
    grouped_yClients[key].append(y)

# Distribute grouped data and load models for each client
grouped_keys = list(grouped_xClients.keys())
num_groups = len(grouped_keys)

for clientID in range(numOfClients):
    if clientID == numOfClients - 1:
        # Assign any remaining groups to the last client
        assigned_keys = grouped_keys[lastLowerBound:]
    else:
        assigned_keys = grouped_keys[lastLowerBound:lastLowerBound + clientDataInterval]

    xClientData = [item for key in assigned_keys for item in grouped_xClients[key]]
    yClientData = [item for key in assigned_keys for item in grouped_yClients[key]]
    print(xClientData)

    xClientsList.append(np.array(xClientData))
    yClientsList.append(np.array(yClientData))
    model=load_model(modelLocation)
    clientsModelList.append(model)

    lastLowerBound += clientDataInterval

# ------- 3. Update clients' model with intial server's deep-model ----------
for clientID in range(numOfClients):
    clientsModelList[clientID].fit(xClientsList[clientID], yClientsList[clientID], epochs=epochs, batch_size=batch_size, verbose=verbose)

start_time = time.time()
process = psutil.Process(os.getpid())
for iterationNo in range(1,numOfIterations+1):
    print("Iteration",iterationNo)
    for clientID in range(numOfClients):
        print("clientID",clientID)
        clientsModelList[clientID].compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
        clientsModelList[clientID].fit(xClientsList[clientID], yClientsList[clientID], epochs=epochs, batch_size=batch_size, verbose=verbose)
        clientWeight=clientsModelList[clientID].get_weights()
        # Find sum of all client's model
        updateServerModel(clientsModelList[clientID], clientWeight)
        firstClientFlag=False
    #Avarage all clients model
    for ind in range(len(deepModelAggWeights)):
        deepModelAggWeights[ind]/=numOfClients

    dw_last=deepModel.get_weights()

    for ind in range(len(deepModelAggWeights)):
        dw_last[ind]=deepModelAggWeights[ind]

    #Update server's model
    deepModel.set_weights(dw_last)
    print("Server's model updated")
    print("Saving model . . .")
    deepModel.save(modelLocation)
    # Servers model is updated, now it can be used again by the clients
    updateClientsModels()
    firstClientFlag=True
    deepModelAggWeights.clear()

    yPredict = deepModel.predict(xTest)
    acc, prec, recall, f1Score= predictTestData(yPredict, yTest)
    accList.append(acc)
    precList.append(prec)
    recallList.append(recall)
    f1List.append(f1Score)
    print("Acc:\n", acc)
    print("Prec:\n", prec)
    print("Recall:\n", recall)
    print("F1-Score:\n", f1Score)

memoryTraining=process.memory_percent()
timeTraining=time.time() - start_time
print("---Memory---",memoryTraining)
print("--- %s seconds (TRAINING)---" % (timeTraining))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

history = deepModel.fit(xServer, yServer, epochs=epochs,
                        validation_data = (xTest,yTest))
                        # callbacks=[early_stopping])

learningAccs=history.history['val_accuracy']
learningLoss=history.history['val_loss']

# resultSaveLocation=root_path+'Results/'+algoName+'_Users_vs_TR_vs_Iterations_vs_AccLossMemTime'+'.csv'
dfSave=pd.DataFrame(columns=['Clients', 'Iterations to converge', 'Accuracy', 'Loss', 'Memory', 'Time'])
dfSaveIndex=0
saveList = [numOfClients, len(learningLoss), learningAccs[len(learningAccs)-1], learningLoss[len(learningLoss)-1], memoryTraining, timeTraining]
dfSave.loc[dfSaveIndex] = saveList

yPredict = deepModel.predict(xTest)
acc, prec, recall, f1Score= predictTestData(yPredict, yTest)

print("Number of users:", numOfClients)
deepModel.save(modelLocation)
print("Epochs:", epochs)
print("BatchSize:", batch_size)
print("Activation:", activationFun, "Optimizer:", optimizerName)

print("Iterations:", numOfIterations)
print("Memory:", memoryTraining)
print("Time:", timeTraining)
print(dfSave)

df_performance_timeRounds = pd.DataFrame(
    {'Accuracy': accList,
     'Precision': precList,
     'Recall': recallList,
     'F1-Score': f1List
    })

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Iteration 1
clientID 0
clientID 1
clientID 2
clientID 3
clientID 4
clientID 5
clientID 6
clientID 7
clientID 8
clientID 9
Server's model updated
Saving model . . .
238/238 [==============================] - 1s 2ms/step
yPredict 7611
Accuracy  : 1.0
Precision : 1.0
Recall : 1.0
f1Score : 1.0
[[7611]]
Acc:
 1.0
Prec:
 1.0
Recall:
 1.0
F1-Score:
 1.0
Iteration 2
clientID 0
clientID 1
clientID 2
clientID 3
clientID 4
clientID 5
clientID 6
clientID 7
clientID 8
clientID 9
Server's model updated
Saving model . . .


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 0s 1ms/step
yPredict 7611
Accuracy  : 1.0
Precision : 1.0
Recall : 1.0
f1Score : 1.0
[[7611]]
Acc:
 1.0
Prec:
 1.0
Recall:
 1.0
F1-Score:
 1.0
Iteration 3
clientID 0
clientID 1
clientID 2
clientID 3
clientID 4
clientID 5
clientID 6
clientID 7
clientID 8
clientID 9
Server's model updated
Saving model . . .


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

238/238 [==============================] - 0s 1ms/step
yPredict 7611
Accuracy  : 1.0
Precision : 1.0
Recall : 1.0
f1Score : 1.0
[[7611]]
Acc:
 1.0
Prec:
 1.0
Recall:
 1.0
F1-Score:
 1.0
Iteration 10
clientID 0
clientID 1
clientID 2
clientID 3
clientID 4
clientID 5
clientID 6
clientID 7
clientID 8
clientID 9
Server's model updated
Saving model . . .


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


238/238 [==============================] - 0s 1ms/step
yPredict 7611
Accuracy  : 1.0
Precision : 1.0
Recall : 1.0
f1Score : 1.0
[[7611]]
Acc:
 1.0
Prec:
 1.0
Recall:
 1.0
F1-Score:
 1.0
---Memory--- 9.195199923915327
--- 2330.5860629081726 seconds (TRAINING)---
Epoch 1/20
429/429 [==============================] - 2s 4ms/step - loss: 5363842678784.0000 - accuracy: 0.0000e+00 - val_loss: 5364523204608.0000 - val_accuracy: 0.0000e+00
Epoch 2/20
429/429 [==============================] - 2s 4ms/step - loss: 5363845824512.0000 - accuracy: 0.0000e+00 - val_loss: 5364523204608.0000 - val_accuracy: 0.0000e+00
Epoch 3/20
429/429 [==============================] - 1s 3ms/step - loss: 5363842154496.0000 - accuracy: 0.0000e+00 - val_loss: 5364523204608.0000 - val_accuracy: 0.0000e+00
Epoch 4/20
429/429 [==============================] - 1s 3ms/step - loss: 5363841630208.0000 - accuracy: 0.0000e+00 - val_loss: 5364523204608.0000 - val_accuracy: 0.0000e+00
Epoch 5/20
429/429 [======================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epochs: 20
BatchSize: 64
Activation: relu Optimizer: Adam
Iterations: 10
Memory: 9.195199923915327
Time: 2330.5860629081726
   Clients  Iterations to converge  Accuracy          Loss  Memory  \
0     10.0                    20.0       0.0  5.364523e+12  9.1952   

          Time  
0  2330.586063  


In [12]:
df_performance_timeRounds

,Accuracy,Precision,Recall,F1-Score
0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0
